<a href="https://colab.research.google.com/github/bohemian-miser/QuantumComputing/blob/main/superposition_of_happy_and_sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install `cirq_google` and `qsimcirq`

try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq-google
    print("installed cirq.")
    import cirq
    import cirq_google

try:
    import qsimcirq
except ImportError:
    print("installing qsimcirq...")
    !pip install --quiet qsimcirq
    print(f"installed qsimcirq.")
    import qsimcirq

# Other modules used in this colab
import matplotlib.pyplot as plt
import time

installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.6 MB/s eta 0:00:00
installed cirq.
installing qsimcirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 13.3 MB/s eta 0:00:00
installed qsimcirq.


# Making a quantum superposition of ASCII art.

Each character is a qubit.
```
|0> -> '_'
|1> -> '#'
```

Here's the art:
```
happy =  "_________________"+\
         "____#_______#____"+\
         "___###_____###___"+\
         "_________________"+\
         "____#_______#____"+\
         "_____#_____#_____"+\
         "______#####______"+\
         "_________________"

sad =    "_________________"+\
         "___###_____###___"+\
         "____#_______#____"+\
         "_________________"+\
         "______#####______"+\
         "_____#_____#_____"+\
         "_____#_____#_____"+\
         "_________________"
```

## Building a quantum superpostion

The goal for our qubits is to have a superposition of happy and sad such that we only get 1 when we measure:
* some always 0 `_` (background)
* some always 1 `#` (elements in both happy and sad)
* the rest will be in an entangled state of happy/sad.

To test this lets make a simplified string of 8 qubits:
* q0 -> 0 (Simulating parts that are always 0).
* q1 -> 1 (Simulating parts that are common to both outputs, e.g eyes).
* q2, q3, q4 -> 000 OR 111 Simulating bits only found in 'Happy'.
* q5, q6, q7 -> 111 OR 000 Simulating bits only found in 'Sad'.

Read off bits right to left: i.e qubit: 76543210.
```
Happy
|00011110>

Sad
|11100010>

```

In [2]:
from sympy import symbols, init_printing
from sympy.physics.quantum import qapply
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.qubit import Qubit
init_printing(use_latex=True)

# Brush up on gates and how they work at https://docs.sympy.org/latest/modules/physics/quantum/gate.html#.
gates = [
    X(1),       # Set the second qubit to 1. Simulating common features of both outputs.
    H(2),       # Choose the first element of the first subset (q2) as the one to be in a superposition of 0/1.
    CNOT(2,5),  # Entangle it with the first element of the second subset (5).
    X(2),       # Swap 0/1 on qubit (2) so that it's now _out of phase_ with (5).
    CNOT(2,3),  # Entangle (2) with the rest of the first set {3,4}.
    CNOT(2,4),
    CNOT(5,6),  # Entangle (5) with the rest of the second set {6,7}.
    CNOT(5,7),
]

# The qubits are in reverse order, i.e 876543210.
current_state =  Qubit('00000000')
print("Initial State:")
display(current_state)

# Iterate through the gates, printing out the state each time.
for i, gate in enumerate(gates):
    current_state = qapply(gate * current_state)
    print(f"\nState after applying {gate}:")
    display(current_state)

Initial State:


❘00000000⟩


State after applying X(1):


❘00000010⟩


State after applying H(2):


...

√2⋅❘00011110⟩   √2⋅❘00100010⟩
───────────── + ─────────────
      2               2      


State after applying CNOT(5,6):


√2⋅❘00011110⟩   √2⋅❘01100010⟩
───────────── + ─────────────
      2               2      


State after applying CNOT(5,7):


√2⋅❘00011110⟩   √2⋅❘11100010⟩
───────────── + ─────────────
      2               2      

```
√2⋅❘00011110⟩   √2⋅❘11100010⟩
───────────── + ─────────────
      2               2      
```
The output above indicates that there is a 50/50 chance of getting:
* Happy:
    * 00011110 (2,3,4) = 111, (5,6,7) = 000
* Sad:
    * 11100010 (2,3,4) = 000, (5,6,7) = 111

We can apply this on a larger scale to our smiley/sad strings.


In [3]:
#@title Circuit diagram
#@markdown In case you are curious, this is what the circuit diagram looks like.
#@markdown Notice that q2 is the only one to perform an X on itself.
#@markdown This is to put it out of phase with q5. The rest of the X gates are
#@markdown controlled (CNOT)'s, are to bring the rest of each set in sync/phase
#@markdown with either q2 and q5.
try:
    import qiskit
except ImportError:
    print("installing qiskit...")
    !pip install --quiet qiskit
    print(f"installed qiskit.")
    import qiskit

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram, circuit_drawer

def create_qiskit_entanglement_circuit(num_qubits, ones, setA, setB):
    # Initialize the circuit with the required number of qubits
    qr = QuantumRegister(num_qubits)
    circuit = QuantumCircuit(qr)

    # Apply X gates to specified qubits
    for idx in ones:
        circuit.x(qr[idx])

    # Entangle sets
    if setA and setB:
        circuit.h(qr[setA[0]])
        circuit.cx(qr[setA[0]], qr[setB[0]])
        circuit.x(qr[setA[0]])
        for idx in setA[1:]:
            circuit.cx(qr[setA[0]], qr[idx])
        for idx in setB[1:]:
            circuit.cx(qr[setB[0]], qr[idx])

    # Add measurement to all qubits to visualize the outcomes
    circuit.measure_all()

    return circuit

# Example usage with dummy indices (update these based on your ASCII art logic)
ones = [1]  # Qubits to be initialized to |1>
setA = [2, 3, 4]  # First set of qubits for the entanglement
setB = [5, 6, 7]  # Second set of qubits for the entanglement
num_qubits = max(ones + setA + setB) + 1  # Determine the number of qubits needed

# Create the circuit
circuit = create_qiskit_entanglement_circuit(num_qubits, ones, setA, setB)

# Visualize the circuit
circuit_drawer = circuit.draw(output='text')
circuit_drawer


installing qiskit...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.9 MB/s eta 0:00:00
installed qiskit.


░ ┌─┐                     
  q0_0: ──────────────────────────░─┤M├─────────────────────
        ┌───┐                     ░ └╥┘┌─┐                  
  q0_1: ┤ X ├─────────────────────░──╫─┤M├──────────────────
        ├───┤     ┌───┐           ░  ║ └╥┘┌─┐               
  q0_2: ┤ H ├──■──┤ X ├──■────■───░──╫──╫─┤M├───────────────
        └───┘  │  └───┘┌─┴─┐  │   ░  ║  ║ └╥┘┌─┐            
  q0_3: ───────┼───────┤ X ├──┼───░──╫──╫──╫─┤M├────────────
               │       └───┘┌─┴─┐ ░  ║  ║  ║ └╥┘┌─┐         
  q0_4: ───────┼────────────┤ X ├─░──╫──╫──╫──╫─┤M├─────────
             ┌─┴─┐          └───┘ ░  ║  ║  ║  ║ └╥┘┌─┐      
  q0_5: ─────┤ X ├──■────■────────░──╫──╫──╫──╫──╫─┤M├──────
             └───┘┌─┴─┐  │        ░  ║  ║  ║  ║  ║ └╥┘┌─┐   
  q0_6: ──────────┤ X ├──┼────────░──╫──╫──╫──╫──╫──╫─┤M├───
                  └───┘┌─┴─┐      ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
  q0_7: ───────────────┤ X ├──────░──╫──╫──╫──╫──╫──╫──╫─┤M├
                       └───┘      ░  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 8/═════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩═
                                     0  1  2  3  4  5  6  7

In [4]:
import cirq

# entangle creates a quantum circuit of 'num_qubits', which produces 1's for the
# qubits in positions defined by 'ones', and then entangles the two sets of positions
# such that only one set will be 1's and the other 0's.
def entangle(num_qubits, ones, setA, setB):
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    for idx in ones:
        # Swap the phase of the 1's from 0, to 1.
        circuit.append(cirq.X(qubits[idx]))

    if setA and setB:  # Check if sets are not empty
        # Choose the first element of the first subset as the one to be in a superposition of 0/1.
        circuit.append(cirq.H(qubits[setA[0]]))
        # Entangle it with the first element of the second subset.
        circuit.append(cirq.CNOT(qubits[setA[0]], qubits[setB[0]]))
        # Set the first element to be out of phase relative to the second set (which it's entangled with).
        circuit.append(cirq.X(qubits[setA[0]]))

        # Entangle the rest of setA with setA[0].
        for idx in setA[1:]:
            circuit.append(cirq.CNOT(qubits[setA[0]], qubits[idx]))

        # Entangle the rest of setB with setB[0].
        for idx in setB[1:]:
            circuit.append(cirq.CNOT(qubits[setB[0]], qubits[idx]))

    circuit.append(cirq.measure(*qubits, key='result'))
    return circuit

def get_matching_ones(smiley, sad):
    return [i for i in range(len(smiley)) if smiley[i] == '#' and sad[i] == '#']

def get_different_sets(smiley, sad):
    setA = [i for i in range(len(smiley)) if smiley[i] != sad[i] and smiley[i] == '#']
    setB = [i for i in range(len(sad)) if smiley[i] != sad[i] and sad[i] == '#']
    return setA, setB

# ASCII Art
smiley = "_________________"+\
         "____#_______#____"+\
         "___###_____###___"+\
         "_________________"+\
         "____#_______#____"+\
         "_____#_____#_____"+\
         "______#####______"+\
         "_________________"

sad =    "_________________"+\
         "___###_____###___"+\
         "____#_______#____"+\
         "_________________"+\
         "______#####______"+\
         "_____#_____#_____"+\
         "_____#_____#_____"+\
         "_________________"

# Calculate qubit positions
ones = get_matching_ones(smiley, sad)
setA, setB = get_different_sets(smiley, sad)
num_qubits = len(smiley)  # Derived from the ASCII grid size (3x3)

# Entangle and simulate
circuit = entangle(num_qubits, ones, setA, setB)
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=5)

print(f"Behold! The power of {num_qubits} qubits!!")
for res in result.measurements['result']:
    # Convert output to ASCII
    output = res
    ascii_output = ''.join(['#' if bit else '_' for bit in output])

    # Reshape ASCII output to grid
    n_lines = 8
    len_lines = int(len(sad)/n_lines)
    for i in range(0, len(smiley), len_lines):
        print(ascii_output[i:i+len_lines])
    print()

Behold! The power of 136 qubits!!
_________________
___###_____###___
____#_______#____
_________________
______#####______
_____#_____#_____
_____#_____#_____
_________________

_________________
____#_______#____
___###_____###___
_________________
____#_______#____
_____#_____#_____
______#####______
_________________

_________________
____#_______#____
___###_____###___
_________________
____#_______#____
_____#_____#_____
______#####______
_________________

_________________
____#_______#____
___###_____###___
_________________
____#_______#____
_____#_____#_____
______#####______
_________________

_________________
____#_______#____
___###_____###___
_________________
____#_______#____
_____#_____#_____
______#####______
_________________



In [5]:
#@title Show the breakdown of how often we measured each state.
import numpy as np
from collections import Counter
def show_measurements(measurements):
    # Flatten the array of measurements into a list of outcomes
    outcomes = measurements[list(measurements.keys())[0]]
    outcomes = [''.join([str(a) for a in x]) for x in outcomes]

    # Count the occurrences of each result (0 or 1)
    result_counts = Counter(outcomes)

    # Calculate the percentage of each outcome
    total_counts = len(outcomes)
    percentages = {state: count / total_counts * 100 for state, count in result_counts.items()}

    # Display the breakdown
    print("State Breakdown and Percentages:")
    for state, percentage in percentages.items():
        print(f"State |{state}⟩: {percentage:.2f}%")

show_measurements(result.measurements)

State Breakdown and Percentages:
State |0000000000000000000011100000111000000010000000100000000000000000000000000011111000000000001000001000000000010000010000000000000000000000⟩: 20.00%
State |0000000000000000000001000000010000000111000001110000000000000000000000001000000010000000001000001000000000001111100000000000000000000000⟩: 80.00%
